<a href="https://colab.research.google.com/github/nikojlieto/firstnn/blob/main/firstnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
#helps move data forwards in the function
import torch.nn.functional as F

In [ ]:
#model class inherits nn.Module
class Model(nn.Module):
  #input layer (antennastate, prxetc, phaserads)
  #hidden layer 1 (some neurons)
  #hidden layer 2 (some neurons)
  #output (x and y)
  def __init__(self, input_features=3, l1 = 10, l2 = 10, output_features=2):
    #instantiate nn.module
    super().__init__()
    out_x = 0
    out_y = 0
    #fc = fully connected
    self.fc1 = nn.Linear(input_features, l1)
    self.fc2 = nn.Linear(l1, l2)
    self.out = nn.Linear(l2, output_features)

    #training data labels - mark categories with likelihood percents
    #argmax(label) = category with most likelihood
    #for no categories feed in x and y coordinates for each sample


  def forward(self, x):
    #relu = rectified linear unit
    #if output < 0, correct to zero
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.out(x)

    return x

In [ ]:
#manual seed for randomization
torch.manual_seed(4)
#create an instance of model
model = Model()

/usr/local/lib/python3.10/dist-packages/torch/nn/init.py:412: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")
